# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [23]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [24]:
data = pd.read_csv("city_weather_data.csv")
data.head()

,City,Latitude,Longitude,Temperature,Max Temperature,Humidity,Cloudiness,Wind,Country,Date
0,Kaitangata,-46.2817,169.8464,60.62,60.62,69,54,5.19,NZ,1635462802
1,Pevek,69.7008,170.3133,1.92,1.92,93,100,4.74,RU,1635462803
2,Kapit,2.0167,112.9333,73.80,73.80,100,100,0.29,MY,1635462805
3,Cascais,38.6979,-9.4215,65.21,66.51,73,100,3.00,PT,1635462805
4,Den Chai,17.9837,100.0522,77.61,77.61,95,91,2.53,TH,1635462806


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [26]:
humidity = data["Humidity"]
m_humidity = humidity.max()
locations = data[["Latitude","Longitude"]]

In [27]:
maps = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity = 8,point_radius = 3)
maps.add_layer(heat_layer)
maps

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [31]:
city_param=data.loc[(data["Temperature"]<85) & (data["Temperature"]>75) & (data["Wind"]<5) & (data["Cloudiness"]<5)]
city_param=city_param.dropna(how='any')
city_param.reset_index(inplace=True)
city_param


,index,City,Latitude,Longitude,Temperature,Max Temperature,Humidity,Cloudiness,Wind,Country,Date
0,215,Half Moon Bay,37.4636,-122.4286,75.06,80.78,54,1,3.00,US,1635463018
1,271,Lázaro Cárdenas,17.9583,-102.2000,82.83,82.83,72,3,4.61,MX,1635462965
2,410,Bosaso,11.2842,49.1816,78.89,78.89,55,0,4.99,SO,1635463187
3,420,Camargo,27.6667,-105.1667,76.21,76.21,11,0,3.62,MX,1635463039
4,547,Qandala,11.4720,49.8728,79.70,79.70,53,1,4.83,SO,1635463319


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
hotel = city_param.loc[:,["City","Country","Latitude","Longitude"]]

hotel["Hotel Name"] = ""

hotel.head()

,City,Country,Latitude,Longitude,Hotel Name
0,Half Moon Bay,US,37.4636,-122.4286,
1,Lázaro Cárdenas,MX,17.9583,-102.2000,
2,Bosaso,SO,11.2842,49.1816,
3,Camargo,MX,27.6667,-105.1667,
4,Qandala,SO,11.4720,49.8728,


In [37]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [38]:
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [50]:
for index, x in hotel.iterrows():
    lat = x["Latitude"]
    lng = x["Longitude"]
    city_name = x["City"]
    
    params["location"] = f"{lat},{lng}"

    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Hotel in {city_name} is {results[0]['name']}.")
        hotel.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("None")
        
   
print("End")

Retrieving Results for Index 0: Half Moon Bay.
Hotel in Half Moon Bay is Oceano Hotel & Spa Half Moon Bay Harbor.
Retrieving Results for Index 1: Lázaro Cárdenas.
Hotel in Lázaro Cárdenas is Hotel Porto.
Retrieving Results for Index 2: Bosaso.
Hotel in Bosaso is Hogato Hotel.
Retrieving Results for Index 3: Camargo.
Hotel in Camargo is Hotel Santa Fe.
Retrieving Results for Index 4: Qandala.
None
End


In [53]:
hotel = hotel.dropna(how='any')
hotel.head()

,City,Country,Latitude,Longitude,Hotel Name
0,Half Moon Bay,US,37.4636,-122.4286,Oceano Hotel & Spa Half Moon Bay Harbor
1,Lázaro Cárdenas,MX,17.9583,-102.2000,Hotel Porto
2,Bosaso,SO,11.2842,49.1816,Hogato Hotel
3,Camargo,MX,27.6667,-105.1667,Hotel Santa Fe
4,Qandala,SO,11.4720,49.8728,


In [57]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel.iterrows()]
locations = hotel[["Latitude", "Longitude"]]

In [59]:
# Add marker layer ontop of heat map
indicator = gmaps.marker_layer(locations,info_box_content=hotel_info)

# Display figure
maps.add_layer(indicator)
maps

Figure(layout=FigureLayout(height='420px'))